In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
input_shape = 28*28  # we receive the size of images in MNIST as 28x28
# as we are having neural network the input shape
# will be 28*28 and we will flatten each image to
# match the input size of model
classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

## Model 1


In [ ]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.nn1 = nn.Linear(input_size, 50)
        self.nn2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.nn1(x))
        x = self.nn2(x)
        return x

In [ ]:
model = NN(input_size=input_shape, num_classes=classes)

In [ ]:
metrics = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Dataset

In [ ]:
train_dataset = datasets.MNIST(root="dataset/", train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False,
                              transform=transforms.ToTensor(),
                              download=True)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

## Training

In [ ]:
for epochs in range(num_epochs):
    for batch_idx, (train, targets) in enumerate(train_loader):
        data = train
        targets = targets  # now we have loaded the data

        # Flatten it to input it into nn
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # now fit the model
        scores = model(data)
        loss = metrics(scores, targets)
        print(scores.shape)
        # CAlculate gradient of the loss wrt the parameters
        # set optimizer's gradients to zero for every batch initially
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()

## Checking accuracy

In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("testing on training data")
    else:
        print("Testing on Testing data")

    num_correct = 0
    num_sample = 0
    model.eval()  # We're telling model to shift to eval mode

    with torch.no_grad():   # we dont want the model to calculate the graidents
        # just the outputs are required as model is already trained
        for x, y in loader:
            x = x.reshape(x.shape[0], -1)
            scores = model(x)
            # scores.max = ([max_values], [indices of max value])
            # Shape = batchsize
            # here indices represent class with max prob
            _, predictions = scores.max(1)
            # we want the class with max probability
            # hence we take max from the last dimension
            # for each image in the batch
            num_correct += (predictions == y).sum()
            # prediction is of shape (batch) and when you compare both
            # you get array of size(batch) with entries of either 1 or 0
            # by summing them you get number of correct predictions

            num_sample += predictions.size(0)

        acc = (float(num_correct) / float(num_sample)*100)
        print(f'Got {float(num_correct)/(float(num_sample))*100:.2f} accuracy')

        # if this was used to check accuracy for every epoch during training
        # add the code
        # model.train()
        # return acc

In [ ]:
check_accuracy(train_loader, model)